# TSP Path Relinking

This notebook contains a basic example of forward and backward **path relinking** in the TSP.

Let's take these two TSP solutions

```python
guiding = [1, 2, 3, 4, 5]
current = [1, 3, 5, 2, 4]
```

>Question: What changes can we make to the `current` solution to `guiding` solution?
> > Changes = simple tweaks: a swap of two city indexes.  For example, [1, 3, 5, 2, 4] => [1, 2, 5, 3, 4].
>> The choices we make to iteratively convert (link) `current` into `guiding` is the path we follow.
>>> PR solutions may be suboptimal.  There when PR is complete we normally use local search to improve the solution.

## imports 

In [1]:
import numpy as np
import sys

## `metapy` imports

For this basic example we only need an to load the data and create an objective function to evaluate various example tours.

In [2]:
# install metapy if running in Google Colab
if 'google.colab' in sys.modules:
    !pip install meta-py

In [3]:
from metapy.tsp import tsp_io as io
from metapy.tsp.euclidean import gen_matrix, plot_tour
from metapy.tsp.objective import OptimisedSimpleTSPObjective

## Load Problem

In [4]:
#load file
file_path = 'https://raw.githubusercontent.com/TomMonks/meta-py/main/data/st70.tsp'

#number of rows in the file that are meta_data
md_rows = 6

#read the coordinates
cities = io.read_coordinates(file_path, md_rows)
matrix = gen_matrix(cities, as_integer=True)

---

### Making moves in a restricted neighbourhood.

The functions below provide basic logic for a single move and enumerating all moves in a path relinking restricted neighbourhood.

In [5]:
def evaluate_neighbour(from_city, current, guiding, obj):
    '''
    Performs a two city swap and returns the cost.
    
    Params:
    ------
    from_city: int
        The city to swap
        
    current: np.ndarray
        The current solution
        
    guiding: np.ndarray
        The guiding solution
        
    obj: Object
        The TSP objective function
        
    Returns:
    -------
    float
    '''
    from_index = np.where(current==from_city)[0][0]
    to_index = np.where(guiding==from_city)[0][0]
    # swap and evaluate
    current[from_index], current[to_index] = current[to_index], current[from_index]
    cost= obj.evaluate(current)
    print(current, cost)
    
    #swap back
    current[from_index], current[to_index] = current[to_index], current[from_index]
    return cost


In [6]:
def next_restricted_neighbour_move(current, guiding, obj):
    '''
    Iteratively search through a restricted neighbourhood based 
    on the guiding solution and greedily selects the best move.
    
    Params:
    -------
    current: np.ndarray
        The current solution
        
    guiding: np.ndarray
        The guiding solution
        
    obj: Object
        The TSP objective function
        
    Returns:
    -------
    (np.ndarray, float)
    (best_solution, best_cost)
    '''
    # cities in the restricted neighbourhood
    swaps = current[current != guiding]
    costs = np.full(len(swaps), -np.inf)
    i = 0
    # is there a way to eliminate the python loop?
    for from_city in swaps:
        # evaluate all swaps in current restricted neighbourhood
        print(f'move {i+1} ', end='=> ')
        costs[i] = evaluate_neighbour(from_city, current, guiding, obj)
        i += 1

    best_index = np.argmax(costs)
    from_city = swaps[best_index]
    
    # get index of cities in current and guiding solutions.
    from_idx = np.where(current==from_city)[0][0]
    to_idx = np.where(guiding==from_city)[0][0]
    # swap and evaluate
    current[from_idx], current[to_idx] = current[to_idx], current[from_idx]
    return current, costs[best_index]

In [7]:
def path_relinking(current, guiding, obj, trunc=None):
    '''
    Path relinking for basic symmetric TSP.
    
    To reverse the path relinking simply swap the current and guiding.
    
    Params:
    ------
    current: np.ndarray
        the current solution that will iteratively be relinked to guiding
        
    guiding: np.ndarray
        the guiding solution.
        
    obj: Object
        TSP objective function
        
    trunc: int, optional (default = None)
        Truncate the search after a number of steps
        
    Returns:
    -------
    (np.ndarray, float)
    (best_solution, best_cost)
    
    '''
    # moves to relink = n - 1: note we don't need to make the final one i.e n-2
    n_moves = int(len(current[current != guiding]) / 2)
    
    # used to truncate path relinking
    if trunc is not None:
        if trunc > n_moves:
            raise ValueError(f'@trunc must be <= moves to relink{n_moves}')
        else:
            n_moves = trunc
        
    costs = []
    solutions = []
    for i in range(n_moves):
        print(f'#### MOVE {i+1}')
        current, cost = next_restricted_neighbour_move(current.copy(), guiding, 
                                                       obj)
        print(f'** Selected: {current}')
        solutions.append(current)
        costs.append(cost)
    
    # return best solution and cost
    best_idx = np.array(costs).argmax()
    return solutions[best_idx], costs[best_idx]

## Let's relink the current solution to the guiding.

We will use **forward** path relinking where we move from the current solution to the guiding solution.

> The opposite would be **backward** relinking where we swap the guiding and current solutions.  That may lead to a different path for relinking.  We also know that good solutions tend to be close together so we might **truncate** our search (partial relinking) in backward PR and save computational effort.
>> In a PR implementation you might want to do BOTH backward and forward search!  That obviously comes at a cost!

In [8]:
# script for forward path relinking
obj = OptimisedSimpleTSPObjective(-matrix)
guiding = np.array([1, 2, 3, 4, 5])
current = np.array([1, 3, 5, 2, 4])
solution, cost = path_relinking(current.copy(), guiding, obj)
print(f'solution after forward PR {solution}: {cost}')

#### MOVE 1
move 1 => [1 5 3 2 4] -147.0
move 2 => [1 3 4 2 5] -138.0
move 3 => [1 2 5 3 4] -134.0
move 4 => [1 3 5 4 2] -117.0
** Selected: [1 3 5 4 2]
#### MOVE 2
move 1 => [1 5 3 4 2] -139.0
move 2 => [1 3 2 4 5] -125.0
move 3 => [1 2 5 4 3] -112.0
** Selected: [1 2 5 4 3]
solution after forward PR [1 2 5 4 3]: -112.0


In [9]:
# script for backward path relinking - swap current and guiding
obj = OptimisedSimpleTSPObjective(-matrix)
current = np.array([1, 2, 3, 4, 5])
guiding = np.array([1, 3, 5, 2, 4])
# also truncate the search
solution, cost = path_relinking(current.copy(), guiding, obj, trunc=1)
print(f'solution after truncated backward PR {solution}: {cost}')

#### MOVE 1
move 1 => [1 4 3 2 5] -142.0
move 2 => [1 3 2 4 5] -125.0
move 3 => [1 2 3 5 4] -121.0
move 4 => [1 2 5 4 3] -112.0
** Selected: [1 2 5 4 3]
solution after truncated backward PR [1 2 5 4 3]: -112.0
